In [1]:
import numpy as np

In [2]:
# Enter the number of layers of your network
num_layers = int(input("Enter the number of layers : "))

# Enter the number of neurons of your network
split_int = lambda x : int(x.split(' '))
num_neurons = input("Enter the number of neurons of each layer in your network")

split_int(num_neurons)

5


In [14]:
# Construct a list that contains the number of neurons of each layer in your network.
structure = input("Enter the number of neurons of each layer in your network").split(' ')
structure = [int(x) for x in structure]
print(structure)    

[2, 5, 3]


In [30]:
# Construct a list that contains all the bias vectors of your network(_n means it has all vectors)
B_n = [np.random.randn(l, 1) for l in structure[1:]]
B_n = np.array(B_n)

print("bias vector of each layer\n")
for B in B_n:
    print(B)
    print()

# Construct a list that contains all the weight matrices of your network(_n means it has all matrices)
W_n = [np.random.randn(l, next_l) for l, next_l in zip(structure[:-1], structure[1:])]
w_n = np.array(W_n)

print("weight matrix of each layer\n")
for W in W_n:
    print(W)
    print()

bias vector of each layer

[[ 2.60503775]
 [-0.75451092]
 [-0.90028959]
 [ 1.42270438]
 [-0.41890072]]

[[-0.88314158]
 [ 1.96014907]
 [ 0.98873152]]

weight matrix of each layer

[[-0.96712273 -0.7144127   0.58876549 -0.78371523  1.10158183]
 [-1.46924907 -0.03249069 -0.10695256  1.71057386  2.76915119]]

[[ 0.78366542  0.96512507 -0.37425481]
 [ 0.8323296   0.2885378   0.50221212]
 [-1.65778657 -1.22604899  0.18092286]
 [-1.43416146 -0.08564885 -0.36229425]
 [ 0.48930521 -1.68884624  1.26006031]]



C:\Users\rubin\AppData\Local\Temp\ipykernel_41980\2924603932.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  B_n = np.array(B_n)
C:\Users\rubin\AppData\Local\Temp\ipykernel_41980\2924603932.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  w_n = np.array(W_n)


##### Forward Pass

In [21]:
# Provide sigmoid and sigmoid_derivative function
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

In [23]:
x = np.array([4, 7])

In [24]:
# Create two lists that contain pre and post activation vector of each layer, respectively
Z_n, A_n = [], []

# Forward pass layer by layer from L=0(First hidden layer) thru L=H(Output layer)
for i, (b, W) in enumerate(zip(B_n, W_n)):
    if i == 0:
        z = np.dot(W.T, x) + b
    else:
        z = np.dot(W.T, a)
    a = sigmoid(z)
    
    Z_n.append(z)
    A_n.append(a)

In [29]:
print(W_n)
print()
print(W_n.T)

[array([[ 0.59716928, -1.62108939, -0.22605826,  0.3382741 , -2.54043146],
       [ 0.88824668, -0.11187268, -0.72703204, -2.72262173,  0.07305009]]), array([[ 0.85730031, -0.56459314, -0.28259055],
       [ 0.9084166 , -1.4537277 ,  0.09349522],
       [-0.55943577,  1.34928798, -0.24278885],
       [-0.63823328, -0.03144898,  0.80522977],
       [-0.11595886, -0.389975  , -0.06478941]])]



AttributeError: 'list' object has no attribute 'T'

In [26]:
for z, a in zip(Z_n, A_n):
    print(z, a)
    print()

[[  6.28437601  -9.58949415  -8.31548516 -20.02728356 -11.97240304]
 [  8.5573902   -7.31647996  -6.04247097 -17.75426937  -9.69938885]
 [  8.83227965  -7.04159051  -5.76758151 -17.47937991  -9.4244994 ]
 [  8.31323346  -7.5606367   -6.28662771 -17.99842611  -9.94354559]
 [  9.62379728  -6.25007287  -4.97606388 -16.68786228  -8.63298177]] [[9.98138252e-01 6.84393509e-05 2.44638285e-04 2.00567824e-09
  6.31609538e-06]
 [9.99807917e-01 6.64055892e-04 2.37005093e-03 1.94723683e-08
  6.13171996e-05]
 [9.99854076e-01 8.73969694e-04 3.11756215e-03 2.56331362e-08
  8.07154850e-05]
 [9.99754810e-01 5.20272884e-04 1.85756832e-03 1.52539687e-08
  4.80343764e-05]
 [9.99933868e-01 1.92659453e-03 6.85387323e-03 5.65657398e-08
  1.78100976e-04]]

[[ 4.50564223e-01 -3.82478611e-04 -1.36168294e-03 -1.12264913e-08
  -3.53483008e-05]
 [-1.08928987e+00 -5.92445791e-04 -2.10829853e-03 -1.73923771e-08
  -5.47616633e-05]
 [ 3.08906625e-01  1.24671941e-04  4.47257572e-04  3.64843704e-09
   1.14908298e-05]] [